In [5]:
%pip install yfinance

  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.9-cp311-cp311-linux_x86_64.whl
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.function_step import step
from sagemaker import get_execution_role, Session
import logging

In [7]:
from steps.fetch import fetch
from steps.preprocess import preprocess
from steps.train import train
from steps.eval import eval

In [8]:
sagemaker_session = Session()
role = get_execution_role()

default_bucket = sagemaker_session.default_bucket()
session = PipelineSession(boto_session=sagemaker_session.boto_session, default_bucket=default_bucket)
pipeline_name = 'stock-pipeline'
default_ticker = "SPY"
years_of_data_to_fetch = 1

In [9]:
# @step(
#     instance_type="ml.m5.large"
# )
# def train(data):
#     # retrieve preprocessed csv data
#     # train model
#     # save model to s3 bucket /model with postfix of date and RMSE in filename
#     logging.info(f'data should be output of preprocess function: {data}')
#     return "stub_train"

In [10]:
# @step(
#     instance_type="ml.m5.large"
# )
# def eval(data):
#     # load current model file and latest
#     # extract model performance from RSME in filename
#     # if model is better than current
#     # pass True to deploy phase (Use conditional to assess when to run deploy)
#     logging.info(f'data should be output of preprocess function: {data}')
#     return False

In [11]:
@step(
    instance_type="ml.m5.large"
)
def deploy(data):
    # if latest_model is different than current
    # update deployment to use latest model
    logging.info(f'data should be output of train function: {data}')
    return "stub_eval"

In [12]:
fetch_result = fetch(default_ticker, years_of_data_to_fetch)
preprocess_result = preprocess(fetch_result)
train_result = train(preprocess_result)
eval_result = eval(train_result)
deploy_result = deploy(eval_result)

pipeline = Pipeline(
    name=pipeline_name,
    steps=[fetch_result, preprocess_result, train_result, eval_result, deploy_result],
    sagemaker_session=session,
)

In [13]:
pipeline.upsert(role)

2025-02-21 05:15:12,543 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-7ee5b9d5-cea8-4ebe-8e3d-59e18036aed9/2025-02-21-05-15-12-480/function
2025-02-21 05:15:12,711 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-7ee5b9d5-cea8-4ebe-8e3d-59e18036aed9/2025-02-21-05-15-12-480/arguments
2025-02-21 05:15:13,033 sagemaker.remote_function INFO     Copied dependencies file at 'requirements.txt' to '/tmp/tmpffeilvfv/requirements.txt'
2025-02-21 05:15:13,065 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-7ee5b9d5-cea8-4ebe-8e3d-59e18036aed9/2025-02-21-05-15-12-480/pre_exec_script_and_dependencies'
2025-02-21 05:15:13,068 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:971173012767:pipeline/stock-pipeline',
 'ResponseMetadata': {'RequestId': 'f41c0992-8742-4f67-9db2-bedbfca3a5cf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f41c0992-8742-4f67-9db2-bedbfca3a5cf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Fri, 21 Feb 2025 05:15:15 GMT'},
  'RetryAttempts': 0}}

In [14]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:971173012767:pipeline/stock-pipeline/execution/l44hpluk9kpk', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x7f61d718f350>)